### This Notebook Provides some Notes about using PyTorch for LinearRegression

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### `Building a Simple Linear Regression Model`

In [2]:
## loading and preparing the dataset
## using only 5 features and 1000 sample(800 for train and 200 for test)
data_reg = datasets.make_regression(n_samples=1000, n_features=5, noise=20, random_state=42)

X = data_reg[0]
y = data_reg[1]

# reshape y
y = y.reshape(-1, 1)

## split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    shuffle=True, random_state=42)

## standardize the dataset
sacler = StandardScaler()
X_train_scaled = sacler.fit_transform(X_train)
X_test_scaled = sacler.transform(X_test)

## convert to float32 then convert to tensors
X_train_scaled = torch.from_numpy(X_train_scaled.astype(np.float32))
X_test_scaled = torch.from_numpy(X_test_scaled.astype(np.float32))

y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

#### Build the Model

In [3]:
## build the Model
input_size = X_train_scaled.shape[1]
output_size = 1   ## taregt

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.lin_reg = nn.Linear(in_features=input_dim, out_features=output_dim)
        
    def forward(self, x):   ## predicting functions
        out = self.lin_reg(x)
        return out
    
lin_reg = LinearRegression(input_size, output_size)  ## create an instance


## Training
print('Training started ____________________________________ \n')

## criteria
learning_rate = 0.001
n_epochs = 100

loss = nn.MSELoss()   ## usin MSE
optimizer = torch.optim.SGD(lin_reg.parameters(), lr=learning_rate)

for epoch in range(n_epochs):
    ## predicting
    y_pred_train = lin_reg(X_train_scaled)
    ## cal the loss
    l = loss(y_train, y_pred_train)
    ## backpropagation
    l.backward()
    ## taka an optimization step
    optimizer.step()
    ## empty the gradients
    optimizer.zero_grad()
    
print('Training finished ____________________________________ \n') 
print()
mse_train = loss(y_pred_train, y_train)    
print('MSE train -->', mse_train.item())

[w, b] = lin_reg.parameters()
print('weight -->', w.detach().numpy())
print('bias -->', b.detach().numpy())

Training started ____________________________________ 

Training finished ____________________________________ 


MSE train --> 3220.382568359375
weight --> [[4.5369606 8.554383  3.3953743 4.686488  3.1131282]]
bias --> [0.27149644]


#### `Evaluation`

In [4]:
## you must stop gradient during inference to not kick the gradient graph during inference
with torch.no_grad():
    y_pred_test = lin_reg(X_test_scaled)
    mse_test = loss(y_test, y_pred_test)
    print('MSE train -->', mse_test.item())

MSE train --> 2976.050048828125


#### Done!